## Setup and Import Libraries

In [1]:
import vertexai
import datetime
import pandas as pd
from utils import authenticate
from google.cloud import bigquery
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')

In [2]:
credentials, PROJECT_ID = authenticate()

In [3]:
REGION = 'us-central1'

In [4]:
vertexai.init(
    project=PROJECT_ID,
    location=REGION,
    credentials=credentials
)

## Using Data Warehouse

In [5]:
bq_client = bigquery.Client(
    project=PROJECT_ID,
    credentials=credentials
)

## Stack Overflow Public Dataset

In [6]:
QUERY_TABLES = """
SELECT
  table_name
FROM
  `bigquery-public-data.stackoverflow.INFORMATION_SCHEMA.TABLES`
"""

In [7]:
query_job = bq_client.query(QUERY_TABLES)

In [8]:
for row in query_job:
    for value in row.values():
        print(value)

posts_answers
users
posts_orphaned_tag_wiki
posts_tag_wiki
stackoverflow_posts
posts_questions
comments
posts_tag_wiki_excerpt
posts_wiki_placeholder
posts_privilege_wiki
post_history
badges
post_links
tags
votes
posts_moderator_nomination


## Data Retrieval

In [9]:
INSPECT_QUERY = """
SELECT
    *
FROM
    `bigquery-public-data.stackoverflow.posts_questions`
LIMIT 3
"""

In [10]:
query_job = bq_client.query(INSPECT_QUERY)

In [11]:
stack_overflow_df = query_job.result().to_arrow().to_pandas()

stack_overflow_df.head()

,id,title,body,accepted_answer_id,answer_count,comment_count,community_owned_date,creation_date,favorite_count,last_activity_date,last_edit_date,last_editor_display_name,last_editor_user_id,owner_display_name,owner_user_id,parent_id,post_type_id,score,tags,view_count
0,73842327,Unable to start ypserv in Ubuntu,<p>I was trying to configure NIS master Server...,NaN,0,0,NaT,2022-09-25 05:56:32.863000+00:00,NaN,2022-09-25 05:56:32.863000+00:00,NaT,None,NaN,None,16454187,None,1,0,linux|ubuntu|linux-kernel|nis,1
1,73629682,Error Running Stable Diffusion from the comman...,<p>I installed Stable Diffusion v1.4 by follow...,NaN,2,0,NaT,2022-09-07 03:10:19.647000+00:00,NaN,2022-09-11 01:53:22.703000+00:00,NaT,None,NaN,None,13025866,None,1,1,python|windows|pytorch|command-line-interface,1281
2,73535922,"Input contains NaN, infinity or a value too la...",<p>my dataframe does not contain NAN or infite...,NaN,0,0,NaT,2022-08-30 00:24:31.970000+00:00,NaN,2022-08-30 00:24:31.970000+00:00,NaT,None,NaN,None,19874740,None,1,0,dataframe,2


## Dealing with Large Datasets
- Large datasets for LLMs often don't fit into memory.
- Select all of the columns and rows of the table posts_questions.

In [12]:
QUERY_ALL = """
SELECT
    *
FROM
    `bigquery-public-data.stackoverflow.posts_questions` q
"""

In [13]:
query_job = bq_client.query(QUERY_ALL)

In [14]:
try:
    stack_overflow_df = query_job.result().to_arrow().to_pandas()
except Exception as e:
    print('The DataFrame is too large to load into memory.', e)

The DataFrame is too large to load into memory. 403 GET https://bigquery.googleapis.com/bigquery/v2/projects/spherical-jetty-465410-u7/queries/f9c1ba32-71d0-4fd0-9dcc-5b265835eddb?maxResults=0&location=US&prettyPrint=false: Response too large to return. Consider specifying a destination table in your job configuration. For more details, see https://cloud.google.com/bigquery/troubleshooting-errors

Location: US
Job ID: f9c1ba32-71d0-4fd0-9dcc-5b265835eddb



## Joining Tables and Query Optimization

When working with (large) data, query optimizing is needed in order to save time and resources.

In [15]:
QUERY = """
SELECT
    CONCAT(q.title, q.body) as input_text,
    a.body AS output_text
FROM
    `bigquery-public-data.stackoverflow.posts_questions` q
JOIN
    `bigquery-public-data.stackoverflow.posts_answers` a
ON
    q.accepted_answer_id = a.id
WHERE
    q.accepted_answer_id IS NOT NULL AND
    REGEXP_CONTAINS(q.tags, "python") AND
    a.creation_date >= "2020-01-01"
LIMIT
    10000
"""

In [16]:
query_job = bq_client.query(QUERY)

In [17]:
stack_overflow_df = query_job.result().to_arrow().to_pandas()

stack_overflow_df.head(2)

,input_text,output_text
0,Django - count number of inline elements in cl...,<p>you can use the reverse lookup with using <...
1,Unexpected output of python3 built-in max for ...,<p>The code for <code>max</code> (and <code>mi...


## Adding Instructions

- Instructions for LLMs have been shown to improve model performance and generalization to unseen tasks (Google, 2022).
- Wihtout the instruction, it is only question and answer. Model might not understand what to do.
- With the instructions, the model gets a guideline as to what task to perform.

In [18]:
INSTRUCTION_TEMPLATE = f"""\
Please answer the following Stackoverflow question on Python. \
Answer it like you are a developer answering Stackoverflow questions.

Stackoverflow question:
"""

In [19]:
stack_overflow_df['input_text_instruct'] = INSTRUCTION_TEMPLATE + ' ' + stack_overflow_df['input_text']

stack_overflow_df.head(2)

,input_text,output_text,input_text_instruct
0,Django - count number of inline elements in cl...,<p>you can use the reverse lookup with using <...,Please answer the following Stackoverflow ques...
1,Unexpected output of python3 built-in max for ...,<p>The code for <code>max</code> (and <code>mi...,Please answer the following Stackoverflow ques...


## Dataset for Tuning

In [21]:
train, evaluation = train_test_split(
    stack_overflow_df,
    test_size=0.2,
    random_state=42
)

## Different Datasets and Flow

- Versioning data is important.
- It allows for reproducibility, traceability, and maintainability of machine learning models.
- Get the timestamp.

In [30]:
date = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

- Generate a jsonl file.
- Name it as tune_data_stack_overflow_python_qa-{date}

In [31]:
cols = ['input_text_instruct','output_text']

tune_jsonl = train[cols].to_json(orient="records", lines=True)

In [32]:
training_data_filename = f"tune_data_stack_overflow_python_qa-{date}.jsonl"

In [33]:
with open(training_data_filename, "w") as f:
    f.write(tune_jsonl)